# AG2: GroupChat (Customized GroupChat flows)

Customized GroupChat flows
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/groupchat/custom-group-chat

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## 1. Callable speaker selection method

### Setting up

In [2]:
# Imports and an LLM configuration for our agents
import os
from autogen import (
    AssistantAgent,
    UserProxyAgent,
    Agent,
    GroupChat,
    GroupChatManager,
)

# Put your key in the OPENAI_API_KEY environment variable
# config_list = {"api_type": "openai", "model": "gpt-4o"}
config_list = {"api_type": "openai", "model": "gpt-4o-mini"}

### Agents and workflow

In [3]:
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
No agent can search the web, use the Engineer to write code to perform any task requiring web access.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=config_list,
)

user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=config_list,
    system_message="""Engineer. You follow the approved plan and MUST write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
Do not install additional packages.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
scientist = AssistantAgent(
    name="Scientist",
    llm_config=config_list,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [4]:
def custom_speaker_selection_func(last_speaker: Agent, groupchat: GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages

    # We'll start with a transition to the planner
    if len(messages) <= 1:
        return planner

    if last_speaker is user_proxy:
        if "Approve" in messages[-1]["content"]:
            # If the last message is approved, let the engineer to speak
            return engineer
        elif messages[-2]["name"] == "Planner":
            # If it is the planning stage, let the planner to continue
            return planner
        elif messages[-2]["name"] == "Scientist":
            # If the last message is from the scientist, let the scientist to continue
            return scientist

    elif last_speaker is planner:
        # Always let the user to speak after the planner
        return user_proxy

    elif last_speaker is engineer:
        if "```python" in messages[-1]["content"]:
            # If the last message is a python code block, let the executor to speak
            return executor
        else:
            # Otherwise, let the engineer to continue
            return engineer

    elif last_speaker is executor:
        if "exitcode: 1" in messages[-1]["content"]:
            # If the last message indicates an error, let the engineer to improve the code
            return engineer
        else:
            # Otherwise, let the scientist to speak
            return scientist

    elif last_speaker is scientist:
        # Always let the user to speak after the scientist
        return user_proxy

    else:
        return "random"

### Create GroupChat and run

In [5]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor],
    messages=[],
    max_round=20,
    # Here we specify our custom speaker selection function
    speaker_selection_method=custom_speaker_selection_func
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=config_list)

In [6]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)

Admin (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Initial Plan

**Objective:** Find the latest paper on GPT-4 from arXiv and explore its potential applications in software development.

#### Step 1: Identify Latest Paper on GPT-4
- **Engineer Task:** Write a script that uses an API to search arXiv for the latest papers related to GPT-4. This script will filter the results to only return the most recent publication.
  
#### Step 2: Analyze Potential Applications
- **Scientist Task:** Review the obtained GPT-4 paper and summarize key findings. Identify potential applications in software development. The analysis will cover:
  - How GPT-4 can be utilized for natural language processing tasks
  - Possible integration of GPT-4 in software product features
  - Use in debugging or code genera

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [7]:
print(chat_result.summary)

### Initial Plan

**Objective:** Find the latest paper on GPT-4 from arXiv and explore its potential applications in software development.

#### Step 1: Identify Latest Paper on GPT-4
- **Engineer Task:** Write a script that uses an API to search arXiv for the latest papers related to GPT-4. This script will filter the results to only return the most recent publication.
  
#### Step 2: Analyze Potential Applications
- **Scientist Task:** Review the obtained GPT-4 paper and summarize key findings. Identify potential applications in software development. The analysis will cover:
  - How GPT-4 can be utilized for natural language processing tasks
  - Possible integration of GPT-4 in software product features
  - Use in debugging or code generation
  - Enhancements in user interface dialogue systems

#### Step 3: Report Results
- **Scientist Task:** Compile the findings of the paper and its applications into a comprehensive report that can be presented to the admin for approval.

### Feedb

In [8]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

2
[{'content': 'Find a latest paper about gpt-4 on arxiv and find its potential applications in software.', 'role': 'assistant', 'name': 'Admin'}, {'content': '### Initial Plan\n\n**Objective:** Find the latest paper on GPT-4 from arXiv and explore its potential applications in software development.\n\n#### Step 1: Identify Latest Paper on GPT-4\n- **Engineer Task:** Write a script that uses an API to search arXiv for the latest papers related to GPT-4. This script will filter the results to only return the most recent publication.\n  \n#### Step 2: Analyze Potential Applications\n- **Scientist Task:** Review the obtained GPT-4 paper and summarize key findings. Identify potential applications in software development. The analysis will cover:\n  - How GPT-4 can be utilized for natural language processing tasks\n  - Possible integration of GPT-4 in software product features\n  - Use in debugging or code generation\n  - Enhancements in user interface dialogue systems\n\n#### Step 3: Repor

## 2. Specified flows using graphs

In [10]:
# ADDED (from previous code) TRY 'transition_graph'
# agent_a = planner
# agent_b = engineer
# agent_c = scientist

In [11]:
# agent_a can transition to any other agent, however, those agents can only transition to agent_a
# transition_graph = {
#     agent_a: [agent_b, agent_c],
#     agent_b: [agent_a],
#     agent_c: [agent_a],
# }

In [12]:
# Imports and LLM Configuration
from autogen import ConversableAgent, GroupChat, GroupChatManager
import random
import os

# Put your key in the OPENAI_API_KEY environment variable
# config_list = {"api_type": "openai", "model": "gpt-4o"}
config_list = {"api_type": "openai", "model": "gpt-4o-mini"}

# Helper function to get an agent from our working list by name
def get_agent_by_name(agents, name) -> ConversableAgent:
    for agent in agents:
        if agent.name == name:
            return agent

In [13]:
# Create an empty directed graph
agents = []
speaker_transitions_dict = {}
secret_values = {}

In [14]:
# Outer loop for prefixes 'A', 'B', 'C'
for prefix in ["A", "B", "C"]:
    # Add 3 nodes with each prefix to the graph using a for loop
    for i in range(3):
        node_id = f"{prefix}{i}"
        secret_value = random.randint(1, 5)  # Generate a random secret value
        secret_values[node_id] = secret_value

        # Create an ConversableAgent for each node (assuming ConversableAgent is a defined class)
        agents.append(
            ConversableAgent(
                name=node_id,
                system_message=f"""Your name is {node_id}.
                    Do not respond as the speaker named in the NEXT tag if your name is not in the NEXT tag. Instead, suggest a relevant team leader to handle the mis-tag, with the NEXT: tag.

                    You have {secret_value} chocolates.

                    The list of players are [A0, A1, A2, B0, B1, B2, C0, C1, C2].

                    Your first character of your name is your team, and your second character denotes that you are a team leader if it is 0.
                    CONSTRAINTS: Team members can only talk within the team, whilst team leader can talk to team leaders of other teams but not team members of other teams.

                    You can use NEXT: to suggest the next speaker. You have to respect the CONSTRAINTS, and can only suggest one player from the list of players, i.e., do not suggest A3 because A3 is not from the list of players.
                    Team leaders must make sure that they know the sum of the individual chocolate count of all three players in their own team, i.e., A0 is responsible for team A only.

                    Keep track of the player's tally using a JSON format so that others can check the total tally. Use
                    A0:?, A1:?, A2:?,
                    B0:?, B1:?, B2:?,
                    C0:?, C1:?, C2:?

                    If you are the team leader, you should aggregate your team's total chocolate count to cooperate.
                    Once the team leader know their team's tally, they can suggest another team leader for them to find their team tally, because we need all three team tallys to succeed.
                    Use NEXT: to suggest the next speaker, e.g., NEXT: A0.

                    Once we have the total tally from all nine players, sum up all three teams' tally, then terminate the discussion using DONE!.
                    """,
                llm_config=config_list,
            )
        )
        speaker_transitions_dict[agents[-1]] = []

    # For each team, create the team's internal transitions (any agent to any agent in a team)
    for source_node in range(3):
        source_id = f"{prefix}{source_node}"
        for target_node in range(3):
            target_id = f"{prefix}{target_node}"
            if source_node != target_node:  # To avoid self-loops
                speaker_transitions_dict[get_agent_by_name(agents, source_id)].append(
                    get_agent_by_name(agents, name=target_id)
                )

In [15]:
# Adding edges between teams
speaker_transitions_dict[get_agent_by_name(agents, "A0")].append(get_agent_by_name(agents, name="B0"))
speaker_transitions_dict[get_agent_by_name(agents, "A0")].append(get_agent_by_name(agents, name="C0"))
speaker_transitions_dict[get_agent_by_name(agents, "B0")].append(get_agent_by_name(agents, name="A0"))
speaker_transitions_dict[get_agent_by_name(agents, "B0")].append(get_agent_by_name(agents, name="C0"))
speaker_transitions_dict[get_agent_by_name(agents, "C0")].append(get_agent_by_name(agents, name="A0"))
speaker_transitions_dict[get_agent_by_name(agents, "C0")].append(get_agent_by_name(agents, name="B0"))

### Create GroupChat and run

In [16]:
group_chat = GroupChat(
    agents=agents,
    messages=[],
    max_round=20,
    allowed_or_disallowed_speaker_transitions=speaker_transitions_dict,
    speaker_transitions_type="allowed",
)

# The GroupChatManager will end the chat when DONE! is received, which agents can say when all counts are done
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=config_list,
    code_execution_config=False,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)

In [17]:
# Start the game from Team A's team leader.
chat_result = agents[0].initiate_chat(
    manager,
    message="""
        There are 9 players in this game, split equally into Teams A, B, C. Therefore each team has 3 players, including the team leader.
        The task is to find out the sum of chocolate count from all nine players. I will now start with my team.
        NEXT: A1
        """,
)

A0 (to chat_manager):


        There are 9 players in this game, split equally into Teams A, B, C. Therefore each team has 3 players, including the team leader.
        The task is to find out the sum of chocolate count from all nine players. I will now start with my team.
        NEXT: A1
        

--------------------------------------------------------------------------------

Next speaker: A1


>>>>>>>> USING AUTO REPLY...
A1 (to chat_manager):

{
  "A0": 0,
  "A1": 5,
  "A2": 0,
  "B0": 0,
  "B1": 0,
  "B2": 0,
  "C0": 0,
  "C1": 0,
  "C2": 0
}

NEXT: B0

--------------------------------------------------------------------------------

Next speaker: A0


>>>>>>>> USING AUTO REPLY...
A0 (to chat_manager):

NEXT: B0

--------------------------------------------------------------------------------

Next speaker: B0


>>>>>>>> USING AUTO REPLY...
B0 (to chat_manager):

NEXT: B0

--------------------------------------------------------------------------------

Next speaker: B1


>>>>>

In [18]:
print(chat_result.summary)

It looks like there has been too much confusion with the sequence. I suggest C0 to address any outstanding issues and help steer the conversation. 

NEXT: C0


In [19]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

20
[{'content': '\n        There are 9 players in this game, split equally into Teams A, B, C. Therefore each team has 3 players, including the team leader.\n        The task is to find out the sum of chocolate count from all nine players. I will now start with my team.\n        NEXT: A1\n        ', 'role': 'assistant', 'name': 'A0'}, {'content': '{\n  "A0": 0,\n  "A1": 5,\n  "A2": 0,\n  "B0": 0,\n  "B1": 0,\n  "B2": 0,\n  "C0": 0,\n  "C1": 0,\n  "C2": 0\n}\n\nNEXT: B0', 'name': 'A1', 'role': 'user'}, {'content': 'NEXT: B0', 'role': 'assistant', 'name': 'A0'}, {'content': 'NEXT: B0', 'name': 'B0', 'role': 'user'}, {'content': "It seems there's a mis-tag, as B0 should not be the next speaker after A1. I'll suggest B1, who can contribute to the tally of their team.\n\nNEXT: B1", 'name': 'B1', 'role': 'user'}, {'content': "It looks like there's a mis-tag. I will suggest B0 to handle this.\n\nNEXT: B0", 'name': 'B2', 'role': 'user'}, {'content': 'It seems like there has been another mis-ta

## More GroupChat examples

- [GroupChat with Customized Speaker Selection Method](https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_groupchat_customized)
- [GroupChat with Coder and Visualization Critic](https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_groupchat_vis)
- [GroupChat with Retrieval-Augmented Generation](https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_groupchat_RAG)
- [Implementing Swarm with a GroupChat](https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_swarm_w_groupchat_legacy)